In [ ]:
import os, sys
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

sys.path.append(os.path.abspath('./diffae/'))

In [ ]:
import numpy as np
import torch
from torchvision.transforms import functional as VF
from PIL import Image
from matplotlib import pyplot

# autoencoder

In [ ]:
# https://drive.google.com/drive/folders/1-5zfxT6Gl-GjxM7z9ZO2AHlB70tfmF6V

In [ ]:
from templates import ffhq256_autoenc, LitModel

In [ ]:
device = 'cuda'
conf = ffhq256_autoenc()
model = LitModel(conf)

# ckpt_path = f'./diffae/checkpoints/{conf.name}/last.ckpt'
# state = torch.load(ckpt_path, map_location='cpu')['state_dict']

ckpt_path = './diffae_metroid_1.pth'
# ckpt_path = './diffae_cg_10.pth'
state = torch.load(ckpt_path, map_location='cpu')

model.load_state_dict(state, strict=False)
model.ema_model.eval()
model.ema_model.to(device);

In [ ]:
# for official ffhq256 dataset

img = Image.open('./diffae/imgs_interpolate/1_a.png')

# for metroid dataset

# img = Image.open('./images/Metroid_Fusion.jpg')
# img = img.crop((128, 128, 128 + 512, 128 + 512))

# img = Image.open('./images/Metroid_Fusion_2.jpg')
# img = img.crop((0, 512, 512, 512 + 512))

# for cg dataset

# img = Image.open('./images/panda4.jpg')
# img = img.crop((256, 50, 512 + 50, 256 + 100))

# img = Image.open('./images/reincarnation.jpg')
# img = img.crop((256 - 100, 0, 512 + 100, 256 + 200))

img = img.resize((256, 256)).convert('RGB')
img

In [ ]:
x = VF.to_tensor(img)
x = x * 2. - 1.
x = x.unsqueeze(0).to(device)

with torch.no_grad():
    
    cond = model.encode(x)
    
    xT = model.encode_stochastic(x, cond, T=250)
    
    pred = model.render(xT, cond, T=20)

cond.shape, xT.shape

In [ ]:
VF.to_pil_image(torch.cat([
    torch.clamp(x[0] * .5 + .5, 0., 1.),
    torch.clamp(xT[0] * .125 + .5, 0., 1.),
    torch.clamp(pred[0], 0., 1.)
], dim=2))

In [ ]:
# for official ffhq256 dataset

img = Image.open('./diffae/imgs_interpolate/1_a.png')
img_a = img

img = Image.open('./diffae/imgs_interpolate/1_b.png')
img_b = img

# for metroid dataset

# img = Image.open('./images/Metroid_Fusion.jpg')
# img = img.crop((128, 128, 128 + 512, 128 + 512))
# img_a = img

# img = Image.open('./images/Metroid_Fusion_2.jpg')
# img = img.crop((0, 512, 512, 512 + 512))
# img_b = img

# for cg dataset

# img = Image.open('./images/panda4.jpg')
# img = img.crop((256, 50, 512 + 50, 256 + 100))
# img_a = img

# img = Image.open('./images/reincarnation.jpg')
# img = img.crop((256 - 100, 0, 512 + 100, 256 + 200))
# img_b = img


img_a = img_a.resize((256, 256)).convert('RGB')
img_b = img_b.resize((256, 256)).convert('RGB')

In [ ]:
x = VF.to_tensor(img_a)
x = x * 2. - 1.
xA = x.unsqueeze(0).to(device)

x = VF.to_tensor(img_b)
x = x * 2. - 1.
xB = x.unsqueeze(0).to(device)

with torch.no_grad():
    
    xR = torch.randn_like(xA)
    condR = torch.randn((1, 512), dtype=xR.dtype, device=xR.device)
    # condR = torch.zeros((1, 512), dtype=xR.dtype, device=xR.device)
    
    condA = model.encode(xA)
    xTA = model.encode_stochastic(xA, condA, T=250)

    xR = torch.mean(xTA, dim=[-2,-1], keepdims=True) + xR * torch.std(xTA, dim=[-2,-1], keepdims=True)
    condR = torch.mean(condA, dim=[-2,-1], keepdims=True) + condR * torch.std(condA, dim=[-2,-1], keepdims=True)
    
    predA = model.render(xTA, condA, T=20)
    predAR = model.render(xR, condA, T=20)
    predAr = model.render(xA, condR, T=20)
    
    condB = model.encode(xB)
    xTB = model.encode_stochastic(xB, condB, T=250)
    predB = model.render(xTB, condB, T=20)
    predBR = model.render(xR, condB, T=20)
    predBr = model.render(xB, condR, T=20)
    
    predAtoB = model.render(xTA, condB, T=20)
    predBtoA = model.render(xTB, condA, T=20)

In [ ]:
VF.to_pil_image(torch.cat([
    torch.cat([
        torch.clamp(xA[0] * .5 + .5, 0., 1.),
        torch.clamp(xTA[0] * .125 + .5, 0., 1.),
        torch.clamp(predA[0], 0., 1.),
        torch.clamp(predAR[0], 0., 1.),
        torch.clamp(predAr[0], 0., 1.),
        torch.clamp(predAtoB[0], 0., 1.),
    ], dim=2),
    torch.cat([
        torch.clamp(xB[0] * .5 + .5, 0., 1.),
        torch.clamp(xTB[0] * .125 + .5, 0., 1.),
        torch.clamp(predB[0], 0., 1.),
        torch.clamp(predBR[0], 0., 1.),
        torch.clamp(predBr[0], 0., 1.),
        torch.clamp(predBtoA[0], 0., 1.)
    ], dim=2)
], dim=1))

# interpolate

In [ ]:
alpha = torch.tensor(np.linspace(0, 1, 10, dtype=np.float32)).to(cond.device)
intp = condA * (1 - alpha[:, None]) + condB * alpha[:, None]

def cos(a, b):
    a = a.view(-1)
    b = b.view(-1)
    a = torch.nn.functional.normalize(a, dim=0)
    b = torch.nn.functional.normalize(b, dim=0)
    return (a * b).sum()

theta = torch.arccos(cos(xTA[0], xTB[0]))
x_shape = xTA[0].shape
intp_x = (torch.sin((1 - alpha[:, None]) * theta) * xTA[0].flatten(0, 2)[None] + torch.sin(alpha[:, None] * theta) * xTB[0].flatten(0, 2)[None]) / torch.sin(theta)
intp_x = intp_x.view(-1, *x_shape)

with torch.no_grad():
    pred = model.render(intp_x, intp, T=20)

In [ ]:
VF.to_pil_image(torch.cat(list(torch.clamp(pred, 0., 1.)), dim=-1))

# sample

In [ ]:
# https://drive.google.com/drive/folders/1-H8WzKc65dEONN-DQ87TnXc23nTXDTYb

In [ ]:
from templates_latent import ffhq256_autoenc_latent, LitModel

In [ ]:
device = 'cuda'

conf = ffhq256_autoenc_latent()
conf.pretrain.path = f'./diffae/checkpoints/ffhq256_autoenc/last.ckpt'
conf.latent_infer_path = f'./diffae/checkpoints/ffhq256_autoenc/latent.pkl'

model = LitModel(conf)

In [ ]:
state = torch.load(f'./diffae/checkpoints/{conf.name}/last.ckpt', map_location='cpu')
model.load_state_dict(state['state_dict'], strict=False)

model.to(device);

In [ ]:
torch.manual_seed(4)
with torch.no_grad():
    imgs = model.sample(8, device=device, T=20, T_latent=200)

In [ ]:
fig, ax = pyplot.subplots(2, 4, figsize=(4*5, 2*5))
ax = ax.flatten()
for i in range(len(imgs)):
    ax[i].imshow(imgs[i].cpu().permute([1, 2, 0]))